# Lấy thông tin log

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
my_api_token_020825 = "a2d2cd5955c1093f173cbe8b11934af2"


def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service (e.g., 'core_course_get_courses').
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    # Tạo tham số mặc định
    default_params = {
        'wstoken': my_api_token_020825,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    
    # Gộp tham số bổ sung (nếu có)
    if params:
        default_params.update(params)
    
    try:
        # Gửi yêu cầu GET tới Moodle API
        response = requests.get(moodle_url, params=default_params)
        
        # Kiểm tra mã trạng thái HTTP
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        # Chuyển đổi phản hồi thành JSON
        data = response.json()
        
        # In phản hồi JSON được định dạng đẹp
        print(f"Phản hồi từ '{wsfunction}':")
        print(json.dumps(data, indent=2, ensure_ascii=False))
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None


def get_log_course_user(userid, courseid, limit = 20):
    data = call_moodle_api('local_userlog_get_logs', {'userid': userid, 'courseid': courseid, 'limit': limit})
    return data


def get_access_count(userid, courseid):
    return call_moodle_api('local_userlog_get_access_count', {
        'userid': userid,
        'courseid': courseid
    })

def get_completion_rate(userid, courseid=0):
    return call_moodle_api('local_userlog_get_completion_rate', {
        'userid': userid,
        'courseid': courseid
    })

def get_time_diffs(userid, courseid):
    return call_moodle_api('local_userlog_get_time_diffs', {
        'userid': userid,
        'courseid': courseid
    })

def get_quiz_attempts(userid):
    return call_moodle_api('local_userlog_get_quiz_attempts', {
        'userid': userid
    })

def get_grade_status(userid, courseid):
    return call_moodle_api('local_userlog_get_grade_status', {
        'userid': userid,
        'courseid': courseid
    })

def get_total_study_time(userid, courseid):
    return call_moodle_api('local_userlog_get_total_study_time', {
        'userid': userid,
        'courseid': courseid
    })

# Hàm ban đầu của bạn (giữ nguyên)
def get_log_course_user(userid, courseid, limit=20):
    return call_moodle_api('local_userlog_get_logs', {
        'userid': userid,
        'courseid': courseid,
        'limit': limit
    })
    
def get_user_object_activity_summary(userid, courseid):
    return call_moodle_api('local_userlog_get_user_object_activity_summary', {
        'userid': userid,
        'courseid': courseid,
    })    


def build_feature_vector(userid, courseid):
    # 1. Access count
    access = call_moodle_api('local_userlog_get_access_count', {'userid': userid, 'courseid': courseid})
    access_dict = {x['objecttable']: x['num_access'] for x in access}
    resource_access = access_dict.get('resource', 0)
    quiz_access = access_dict.get('quiz', 0)
    hvp_access = access_dict.get('hvp', 0)
    forum_access = access_dict.get('forum', 0)

    # 2. Completion rate
    completion = call_moodle_api('local_userlog_get_completion_rate', {'userid': userid, 'courseid': courseid})
    completion_rate = completion.get('completion_rate', 0.0)

    # 3. Time diffs
    time_diffs_raw = call_moodle_api('local_userlog_get_time_diffs', {'userid': userid, 'courseid': courseid})
    time_diffs = [x['time_diff'] for x in time_diffs_raw if x['time_diff'] is not None and x['time_diff'] > 0]
    avg_time_diff = np.mean(time_diffs) if time_diffs else 0
    max_time_diff = max(time_diffs) if time_diffs else 0

    # 4. Quiz attempts
    quiz_attempts_raw = call_moodle_api('local_userlog_get_quiz_attempts', {'userid': userid})
    total_quiz_attempts = sum([q['num_attempts'] for q in quiz_attempts_raw])

    # 5. Grade status
    grade_status_raw = call_moodle_api('local_userlog_get_grade_status', {'userid': userid, 'courseid': courseid})
    pass_count = sum(1 for g in grade_status_raw if g['status'] == 'Pass')
    fail_count = sum(1 for g in grade_status_raw if g['status'] == 'Fail')
    pass_rate = pass_count / (pass_count + fail_count) if (pass_count + fail_count) > 0 else 0.0

    # 6. Total time
    total_time_raw = call_moodle_api('local_userlog_get_total_study_time', {'userid': userid, 'courseid': courseid})
    total_time_spent = total_time_raw.get('total_time_spent', 0)

    # Build feature vector
    vector = {
        'userid': userid,
        'courseid': courseid,
        'resource_access': resource_access,
        'quiz_access': quiz_access,
        'hvp_access': hvp_access,
        'forum_access': forum_access,
        'completion_rate': round(completion_rate, 4),
        'avg_time_diff': round(avg_time_diff, 2),
        'max_time_diff': max_time_diff,
        'total_quiz_attempts': total_quiz_attempts,
        'pass_rate': round(pass_rate, 4),
        'total_time_spent': total_time_spent
    }

    return vector


In [5]:
if __name__ == "__main__":
    data = get_log_course_user(4, 5, 20) 

Phản hồi từ 'local_userlog_get_logs':
[
  {
    "id": 4519,
    "eventname": "\\core\\event\\course_viewed",
    "component": "core",
    "action": "viewed",
    "target": "course",
    "objecttable": null,
    "objectid": null,
    "crud": "r",
    "edulevel": 2,
    "contextid": 79,
    "contextlevel": 50,
    "contextinstanceid": 5,
    "userid": 4,
    "courseid": 5,
    "relateduserid": null,
    "anonymous": 0,
    "other": "null",
    "timecreated": 1753955295,
    "origin": "web",
    "ip": "0:0:0:0:0:0:0:1",
    "realuserid": null
  },
  {
    "id": 4069,
    "eventname": "\\core\\event\\section_viewed",
    "component": "core",
    "action": "viewed",
    "target": "section",
    "objecttable": "course_sections",
    "objectid": 39,
    "crud": "r",
    "edulevel": 2,
    "contextid": 79,
    "contextlevel": 50,
    "contextinstanceid": 5,
    "userid": 4,
    "courseid": 5,
    "relateduserid": null,
    "anonymous": 0,
    "other": "null",
    "timecreated": 1753850768,
   

In [41]:
userid = 4
courseid = 5

access_counts = get_access_count(userid, courseid)
completion = get_completion_rate(userid, courseid)
time_diffs = get_time_diffs(userid, courseid)
quiz_attempts = get_quiz_attempts(userid)
grade_status = get_grade_status(userid, courseid)
study_time = get_total_study_time(userid, courseid)
user_object_activity_summary = get_user_object_activity_summary(userid, courseid)


Phản hồi từ 'local_userlog_get_access_count':
[
  {
    "objecttable": "resource",
    "num_access": 3
  },
  {
    "objecttable": "hvp",
    "num_access": 3
  },
  {
    "objecttable": "quiz",
    "num_access": 11
  }
]
Phản hồi từ 'local_userlog_get_completion_rate':
{
  "userid": 4,
  "courseid": 5,
  "completed": 2,
  "total": 3,
  "completion_rate": 0.6667
}
Phản hồi từ 'local_userlog_get_time_diffs':
[
  {
    "logid": 3905,
    "timecreated": 1753847184,
    "time_diff": 0
  },
  {
    "logid": 3907,
    "timecreated": 1753847188,
    "time_diff": 4
  },
  {
    "logid": 3908,
    "timecreated": 1753847207,
    "time_diff": 19
  },
  {
    "logid": 3909,
    "timecreated": 1753847228,
    "time_diff": 21
  },
  {
    "logid": 3910,
    "timecreated": 1753847230,
    "time_diff": 2
  },
  {
    "logid": 3912,
    "timecreated": 1753847335,
    "time_diff": 105
  },
  {
    "logid": 3913,
    "timecreated": 1753847337,
    "time_diff": 2
  },
  {
    "logid": 3914,
    "timecreate

In [10]:
# Ví dụ: phân tích nhiều học viên
userids = [4]  # danh sách học viên
courseid = 5
all_vectors = []

for uid in userids:
    vec = build_feature_vector(uid, courseid)
    all_vectors.append(vec)

# Lưu thành file CSV
df = pd.DataFrame(all_vectors)
df.to_csv("feature_vectors.csv", index=False)

print("✅ Đã lưu vector đặc trưng vào feature_vectors.csv")

Phản hồi từ 'local_userlog_get_access_count':
[
  {
    "objecttable": "resource",
    "num_access": 2
  },
  {
    "objecttable": "hvp",
    "num_access": 3
  },
  {
    "objecttable": "quiz",
    "num_access": 7
  }
]
Phản hồi từ 'local_userlog_get_completion_rate':
{
  "userid": 4,
  "courseid": 5,
  "completed": 2,
  "total": 3,
  "completion_rate": 0.6667
}
Phản hồi từ 'local_userlog_get_time_diffs':
[
  {
    "logid": 3905,
    "timecreated": 1753847184,
    "time_diff": 0
  },
  {
    "logid": 3907,
    "timecreated": 1753847188,
    "time_diff": 4
  },
  {
    "logid": 3908,
    "timecreated": 1753847207,
    "time_diff": 19
  },
  {
    "logid": 3909,
    "timecreated": 1753847228,
    "time_diff": 21
  },
  {
    "logid": 3910,
    "timecreated": 1753847230,
    "time_diff": 2
  },
  {
    "logid": 3912,
    "timecreated": 1753847335,
    "time_diff": 105
  },
  {
    "logid": 3913,
    "timecreated": 1753847337,
    "time_diff": 2
  },
  {
    "logid": 3914,
    "timecreated

In [16]:
import random

def generate_realistic_users(base_row, num_users=1000):
    fake_data = []
    for i in range(num_users):
        new_user = base_row.copy()
        new_user['userid'] = 1000 + i  # giả định userid 1000+

        new_user['resource_access'] = max(0, int(random.gauss(mu=base_row['resource_access'], sigma=1.5)))
        new_user['quiz_access'] = max(0, int(random.gauss(mu=base_row['quiz_access'], sigma=3)))
        new_user['hvp_access'] = max(0, int(random.gauss(mu=base_row['hvp_access'], sigma=1.2)))
        new_user['forum_access'] = random.randint(0, 5)

        new_user['completion_rate'] = round(min(1.0, max(0.0, random.gauss(mu=base_row['completion_rate'], sigma=0.15))), 4)
        new_user['avg_time_diff'] = max(0, round(random.gauss(mu=base_row['avg_time_diff'], sigma=1500), 2))
        new_user['max_time_diff'] = max(0, int(random.gauss(mu=base_row['max_time_diff'], sigma=20000)))

        new_user['total_quiz_attempts'] = max(0, int(random.gauss(mu=base_row['total_quiz_attempts'], sigma=1)))
        new_user['pass_rate'] = round(min(1.0, max(0.0, random.gauss(mu=base_row['pass_rate'], sigma=0.2))), 4)
        new_user['total_time_spent'] = max(0, int(random.gauss(mu=base_row['total_time_spent'], sigma=800)))

        fake_data.append(new_user)
    return pd.DataFrame(fake_data)

In [18]:
df_real = pd.read_csv("feature_vectors.csv")
df_fake = generate_realistic_users(df_real.iloc[0], num_users=1000)

df_all = pd.concat([df_real, df_fake], ignore_index=True)
df_all.to_csv("feature_vectors_expanded.csv", index=False)

print("✅ Đã tạo 1000 user sát thực tế và lưu vào feature_vectors_expanded.csv")

✅ Đã tạo 1000 user sát thực tế và lưu vào feature_vectors_expanded.csv


## Get log cần thiết để đưa vào Qtable

In [3]:
def build_user_log_summary(userid, courseid, user_cluster=None):
    """
    Tổng hợp log học tập để tạo đầu vào cho Q-table.
    """
    # 1. Lấy danh sách log gần đây
    logs = get_user_object_activity_summary(userid, courseid)
    
    # 2. Xác định loại hoạt động cuối cùng (resource, quiz, hvp)
    last_type = 'none'
    last_result = 'none'
    
    if logs:
        for log in (logs):
            obj = log.get('objecttable')
            if obj in ['quiz', 'resource', 'hvp']:
                last_type = obj
                break

    # 3. Nếu là quiz, thì xem thử lần gần nhất có pass hay fail
    if last_type == 'quiz':
        grade_data = get_grade_status(userid, courseid)
        # Lấy bài kiểm tra gần nhất có điểm
        if grade_data:
            last_grade = grade_data[-1]
            last_result = last_grade.get('status', 'none').lower()
    elif last_type in ['resource', 'hvp']:
        last_result = 'done'
    else:
        last_result = 'none'

    # 4. Completion rate
    completion = get_completion_rate(userid, courseid)
    completion_rate = round(completion.get('completion_rate', 0.0), 4)

    # 5. Pass rate & số lượng quiz điểm thấp
    grade_data = get_grade_status(userid, courseid)
    pass_count = sum(1 for g in grade_data if g['status'] == 'Pass')
    fail_count = sum(1 for g in grade_data if g['status'] == 'Fail')
    pass_rate = pass_count / (pass_count + fail_count) if (pass_count + fail_count) > 0 else 0.0

    # Số lượng quiz điểm thấp (giả sử finalgrade < 5 là thấp)
    low_score_quiz_count = sum(1 for g in grade_data if g.get('finalgrade', 10) < 5)

    # 6. Cluster nếu có truyền vào
    cluster = user_cluster if user_cluster is not None else -1

    # 7. Build dictionary kết quả
    summary = {
    'userid': userid,
    'courseid': courseid,
    'cluster': cluster,
    'last_type': last_type,
    'last_result': last_result,
    'completion_rate': completion_rate,
    'pass_rate': round(pass_rate, 4),
    'low_score_quiz_count': low_score_quiz_count
}

    return summary

In [5]:
import pandas as pd

def get_user_cluster(userid, csv_path="feature_vectors_clustered.csv"):
    """
    Trả về cluster của một user dựa trên userid.

    Args:
        userid (int or float): ID của người dùng cần tra cứu.
        csv_path (str): Đường dẫn đến file CSV chứa dữ liệu phân cụm.

    Returns:
        int: Số thứ tự cụm (cluster) của user, hoặc None nếu không tìm thấy.
    """
    df = pd.read_csv(csv_path)

    # Tìm dòng có userid tương ứng
    user_row = df[df['userid'] == userid]

    if not user_row.empty:
        return int(user_row.iloc[0]['cluster'])
    else:
        print(f"❌ Không tìm thấy userid: {userid}")
        return None

In [9]:
def save_summary_to_csv(summary, filename="user_log_summary.csv", mode='a'):
    """
    Ghi log summary của một user vào file CSV. 
    Nếu file chưa tồn tại, sẽ tạo mới với header.
    """
    df = pd.DataFrame([summary])
    
    # Nếu ghi nối (append), kiểm tra xem có cần header không
    write_header = False
    try:
        with open(filename, 'r') as f:
            pass
    except FileNotFoundError:
        write_header = True
    
    df.to_csv(filename, mode=mode, index=False, header=write_header)

user_cluster = get_user_cluster(userid=4)
summary = build_user_log_summary(userid=4, courseid=5, user_cluster=user_cluster)
save_summary_to_csv(summary)

Phản hồi từ 'local_userlog_get_user_object_activity_summary':
[
  {
    "objecttable": "quiz",
    "objectid": 3,
    "latest_time": 1754209267
  },
  {
    "objecttable": "quiz_attempts",
    "objectid": 6,
    "latest_time": 1753847371
  },
  {
    "objecttable": "hvp",
    "objectid": 1,
    "latest_time": 1754209158
  },
  {
    "objecttable": "course_sections",
    "objectid": 38,
    "latest_time": 1753848739
  },
  {
    "objecttable": "resource",
    "objectid": 10,
    "latest_time": 1754119656
  },
  {
    "objecttable": "grade_grades",
    "objectid": 26,
    "latest_time": 1753847362
  },
  {
    "objecttable": "course_modules_completion",
    "objectid": 5,
    "latest_time": 1753848653
  },
  {
    "objecttable": "course",
    "objectid": 5,
    "latest_time": 1753847184
  }
]
Phản hồi từ 'local_userlog_get_grade_status':
[
  {
    "itemname": null,
    "finalgrade": 20,
    "gradepass": 0,
    "status": "Pass"
  },
  {
    "itemname": "abc",
    "finalgrade": 0,
    "gra